# read file and create dictionary

In [102]:
import os, glob, math

class email:
    # represents each email 
    def __init__(self):
        self.classType = 'undefined'
        self.words = []
        self.vector = []
        self.predict = 'undefined'
    def openFile(self, classType, filePath):
        self.classType = classType
        filterList = [',', '.', '!', '?', ';']
        with open(filePath) as myFile:
            data = myFile.read()
            words = set()
            words = data.split()
            for word in words:
                word.lower()
            words = set(words)
#             for char in filterList:
#                 if(char in words):
#                     words.remove(char)
            self.words = words
    def createVector(self, vocab):
        for word in vocab:
            if(word in self.words):
                self.vector.append(1)
            else:
                self.vector.append(0)

class dictionary:
    # word dictionary where words are stored and 
    def __init__(self):
#         self.spamWords = []
#         self.hamWords = []
        self.words = []
        self.spamCount = 0
        self.hamCount = 0
    def update(self, mail):
        tempWordBag = []
        for word in self.words:
            tempWordBag.append(word[0])
        mailWords = mail.words
        if(mail.classType == 'ham'):
            self.hamCount += 1
        else:
            self.spamCount += 1
        for mword in mailWords:
            if(mword in tempWordBag):
                index = tempWordBag.index(mword)
                sword = self.words[index]
                if(mail.classType == 'ham'):
                    sword[1] += 1
                else:
                    sword[2] += 1
            else:
                sword = [mword, 0, 0]
                if(mail.classType == 'ham'):
                    sword[1] += 1
                else:
                    sword[2] += 1
                self.words.append(sword)
                
#         if(mail.classType == 'ham'):
#             self.hamCount += 1
#             for word in self.hamWords:
#                 tempWordBag.append(word[0])
#             mailWords = mail.words
#             for mword in mailWords:
#                 if(mword in tempWordBag):
#                     index = tempWordBag.index(mword)
#                     sword = self.hamWords[index]
#                     sword[1] += 1
#                 else:
#                     sword = [mword, 1]
#                     self.hamWords.append(sword)
#         else:
#             self.spamCount += 1
#             for word in self.spamWords:
#                 tempWordBag.append(word[0])
#             mailWords = mail.words
#             for mword in mailWords:
#                 if(mword in tempWordBag):
#                     index = tempWordBag.index(mword)
#                     sword = self.spamWords[index]
#                     sword[1] += 1
#                 else:
#                     sword = [mword, 1]
#                     self.spamWords.append(sword)
    def display(self):
        print('spam count: ', self.spamCount)
        print('ham count: ', self.hamCount)
        for word in self.words:
            print(word)
#         print('spam words')
#         for word in self.spamWords:
#             print(word)
#         print('ham words')
#         for word in self.hamWords:
#             print(word)

path = 'emails/bare'
# os.path = returns current directory
path = os.path.join(os.getcwd(), path)

wordDict = dictionary()
mailList = []
# adjust value in range if you want to add more parts
for part in enumerate(range(10), 1):
    folder = os.path.join(path, 'part'+str(part[0]))
    for filePath in glob.glob(os.path.join(folder, '*.txt')):
        mail = email()
        if('spmsga' in filePath):
            mail.openFile('spam', filePath)
        else:
            mail.openFile('ham', filePath)
        mailList.append(mail)
        wordDict.update(mail)


# calculate mutual information score

In [103]:
class mutualInformation:
    def __init__(self, countHam, countSpam):
        self.probHam = 1.0 * countHam / (countHam+countSpam)
        self.probSpam = 1.0 * countSpam / (countHam+countSpam)
        self.words = []
    def probOfWord(self, words):
        countH = 0
        countS = 0
        for word in words:
            countH += word[1]
            countS += word[2]
        count = countH+countS
        for tempWord in words:
            # probability of a word appearing
            wordprob = 1.0 * (tempWord[1]+tempWord[2])/ count
            # calculate the probability of word given a class(ham)
            hwordprob = 1.0 * self.probHam * (1.0 * tempWord[1]/countH)
            # calculate the probability of word given a class(ham), P(X,C)*log(P(X,C)/(P(X)*P(C)))
            if(hwordprob == 0):
                htemp = 0
            else:
                htemp = hwordprob * math.log(hwordprob/(wordprob*self.probHam))
            # calculate the probability of word given a class(spam)
            swordprob = 1.0 * self.probSpam * (1.0 * tempWord[2]/countS)
            # calculate the probability of word given a class(spam), P(X,C)*log(P(X,C)/(P(X)*P(C)))
            if(swordprob == 0):
                stemp = 0
            else:
                stemp = swordprob * math.log(swordprob/(wordprob*self.probSpam))
            temp = htemp + stemp
            word = [tempWord[0], temp]
            self.words.append(word)
        # sort words according to miscore, descending
        self.words.sort(key = self.getmi, reverse = True)
        
    def getmi(self, word):
        # function to return only the miscore
        return word[1]   
        
    def getWords(self, number):
        # get n number of words that have the highest miscore
        vocab = []
        for index in range(number):
            word = self.words[index]
            vocab.append(word[0])
        return vocab
        

    
mi = mutualInformation(wordDict.hamCount, wordDict.spamCount)
# print('probability of ham: %f' %mi.probHam)
# print('probability of spam: %f' %mi.probSpam)
# calculate probability of a word
words = wordDict.words

mi.probOfWord(words)
# get 50 words with the highest mi
numOfWords = [50, 100, 200, 300]
vocab = mi.getWords(numOfWords[0])
for voc in vocab:
    print(voc)
    

language
university
linguistics
free
!
money
remove
languages
english
service
thousands
linguistic
receive
your
income
papers
get
de
cost
%
cash
000
best
million
edu
product
100
company
tax
business
$
marketing
sales
mailing
offer
credit
today
theory
earn
@
hundreds
want
our
you
checks
visa
removed
syntax
mailings
speakers


# create vector for each email

In [104]:
for mail in mailList:
    # create vector list based on reduced vocabulary
    mail.createVector(vocab)
#     print(mail.vector)

# naive bayesian classifier

In [105]:
# did not commit starting here
class bayesian:
    def __init__(self):
        self.spamWordCount = 0
        self.hamWordCount = 0
        self.spamWordProb = []
        self.hamWordProb = []
        self.accuracy = 0
        self.wAccuracy = 0
        self.bwAccuracy = 0
        self.srecall = 0
        self.sprecision = 0
        self.trc = 0

    def getWordCount(self, mailList):
        for mail in mailList:
            if(mail.classType == 'ham'):
                self.hamWordCount += sum(mail.vector)
            else:
                self.spamWordCount += sum(mail.vector)
                
    def getWordProb(self, vocab, mailList, mi, spamCount, hamCount):
        swcount = []
        hwcount = []
        for word in vocab:
            scount = 0
            hcount = 0
            for mail in mailList:
                if(word in mail.words):
                    if(mail.classType == 'ham'):
                        hcount += 1
                    else:
                        scount += 1
            swcount.append(scount)
            hwcount.append(hcount)
        # get probability of each word in spam class
        for word in swcount:
            prob = 1.0 * word/spamCount
            self.spamWordProb.append(prob)
        # get probability of each word in ham class
        for word in hwcount:
            prob = 1.0 * word/hamCount
            self.hamWordProb.append(prob)
            
    def predict(self, mailList, vocab, mi, lamb):
        threshold = 1.0 * lamb /(1+lamb)
        for mail in mailList:
            sprodProb = 1.0
            hprodProb = 1.0
            for mword in mail.words:
                if(mword in vocab):
                    index = vocab.index(mword)
                    swprob = self.spamWordProb[index]
                    hwprob = self.hamWordProb[index]
                    if(swprob > 0):
                        sprodProb *= swprob
                    else:
                        sprodProb *= (1-swprob)
                    if(hwprob > 0):
                        hprodProb *= hwprob
                    else:
                        hprodProb *= (1-hwprob)
            sclassProb = (mi.probSpam*sprodProb)
            hclassProb = (mi.probHam*hprodProb)
            denominator = sclassProb+hclassProb
            # probability that mail is spam
            sclassProb = sclassProb / denominator
            # probability that mail is not spam
            hclassProb = hclassProb / denominator
            # choose prediction based on whose probability is greater
            if(sclassProb>threshold):
                mail.predict = 'spam'
            else:
                mail.predict = 'ham'
                
    def calculatePerformance(self, mailList, lamb):
        countTP, countFP, countTN, countFN = 0, 0, 0, 0
        countHam, countSpam = 0, 0
        # true positive is spam that was predicted to be spam
        for mail in mailList:
            if(mail.classType == 'ham'):
#                 countHam += 1
                if(mail.predict == 'ham'):
                    countTN += 1
                else:
                    countFP += 1
            else:
#                 countSpam += 1
                if(mail.predict == 'ham'):
                    countFN += 1
                else:
                    countTP += 1
        print(countTP, countFP, countTN, countFN)
        # calculate the performance measures
        self.accuracy = 1.0 * (countTP+countTN)/(countTP+countTN+countFP+countFN)
        self.wAccuracy = 1.0 * (lamb*countTP+countTN)/(lamb*(countTN+countFP)+countTP+countFN)
        self.bwAccuracy = 1.0 * (lamb*(countTN+countFP))/(lamb*(countTN+countFP)+countTP+countFN)
        self.srecall = 1.0 * countTP/(countTP+countFN)
        self.sprecision = 1.0 * countTP/(countTP+countFP)
        self.trc = 1.0 * (countTP+countFN)/(lamb*countFP+countFN)
            
bay = bayesian()
bay.getWordCount(mailList)
# print(bay.spamWordCount)
# print(bay.hamWordCount)

bay.getWordProb(vocab, mailList, mi, wordDict.spamCount, wordDict.hamCount)
# print(bay.spamWordProb)
# print(bay.hamWordProb)
lamb = [1, 9, 999]
tempLambda = lamb[0]
bay.predict(mailList, vocab, mi, tempLambda)
# for mail in mailList:
#     print('actual class: %s class prediction: %s' %(mail.classType, mail.predict))
bay.calculatePerformance(mailList, tempLambda)
print('accuracy: %f' %bay.accuracy)
print('spam recall: %f' %bay.srecall)
print('spam precision: %f' %bay.sprecision)
print('weighted accuracy: %f' %bay.wAccuracy)
print('baseline weighted accuracy %f' %bay.bwAccuracy)
print('TRC: %f' %bay.trc)

(161, 550, 2177, 5)
accuracy: 0.808158
spam recall: 0.969880
spam precision: 0.226442
weighted accuracy: 0.808158
baseline weighted accuracy 0.942620
TRC: 0.299099
